In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

### Import the data

In [2]:
df = pd.read_csv("cleaned_bank_data.csv")

In [3]:
# df.sum()

In [4]:
#Dropping one column each from categorical variables to avoid redunduncy
# df = df.drop([
#  'poutcome_success',
#  'job_unemployed',
#  'marital_divorced',
#  'month_apr',
#  'day_of_week_fri'],axis=1)
df = df.drop('Unnamed: 0',axis=1)
features = df.drop('y',axis=1)

### Split the data into training and test sets for final testing

In [5]:
X, Xtest, y, ytest = train_test_split(features.values, df['y'].values,random_state=5,test_size=0.2)

### Tuning

In [6]:
def cv_score(clf, x, y, score_func=accuracy_score): #Apply k-fold cross velidation
    result = 0
    nfold = 5
    for train, test in KFold(nfold).split(x): # split data into train/test groups, 5 times
        clf.fit(x[train], y[train]) # fit
        result += score_func(clf.predict(x[test]), y[test]) # evaluate score function on held-out data
    return result / nfold # average

In [7]:
clf = LogisticRegression()
score = cv_score(clf, X, y)
print(score)

0.908455571172


#### Tune C

In [8]:
#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]
max_score = 0
# your turn
for C in Cs:
    clf = LogisticRegression(C=C)
    score = cv_score(clf, X, y)
#     print (score)
    if score > max_score:
        max_score = score
        Cfinal = C
print ("Maximum score of {} is achieved at C = {}".format(max_score,Cfinal))

Maximum score of 0.9085168644608128 is achieved at C = 100


In [9]:
# your turn
clf = LogisticRegression(C=Cfinal)
clf.fit(X, y)
print(accuracy_score(clf.predict(Xtest), ytest))
#clf.score(Xtestlr,ytestlr)

0.908801176759


Attempt 1 : No feature scaling, including all the features - 0.909168913949

Attempt 2 : Removed the features containing redundant information (derived from categorical variables) - 0.908188281442

Attempt 3 : Attempt 1 + Feature Scaling - 0.910026967394

Attempt 4 : Attempt 2 + Feature Scaling - 0.910026967394

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, clf.predict(Xtest))
cm

array([[7096,  143],
       [ 601,  318]])